<a href="https://colab.research.google.com/github/2303A51111/B3_PFDS_1111/blob/main/project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Load dataset
df = pd.read_csv("college_student_placement_dataset.csv")

# Preprocessing
# Encode categorical variables
for col in df.select_dtypes(include=['object']).columns:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# Features and target
X = df.drop("Placement", axis=1, errors="ignore")
y = df["Placement"]

# If target is continuous percentage, discretize into classes (e.g., High/Medium/Low)
# Uncomment if needed
# y = pd.qcut(y, q=3, labels=["Low", "Medium", "High"])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply PCA (optional, keep 95% variance)
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": GaussianNB(),
    "SVM": SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

# Train & Evaluate
results = {}
for name, model in models.items():
    model.fit(X_train_pca, y_train)
    preds = model.predict(X_test_pca)
    acc = accuracy_score(y_test, preds)
    report = classification_report(y_test, preds, zero_division=0)

    # Cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_pca, y_train, cv=kf)

    results[name] = {
        "Accuracy": acc,
        "CV Mean": np.mean(cv_scores),
        "Report": report
    }

# Print results
for model_name, metrics in results.items():
    print(f"\n===== {model_name} =====")
    print(f"Test Accuracy: {metrics['Accuracy']:.4f}")
    print(f"Cross-Val Accuracy: {metrics['CV Mean']:.4f}")
    print("Classification Report:\n", metrics['Report'])


===== Logistic Regression =====
Test Accuracy: 0.9025
Cross-Val Accuracy: 0.9009
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94      1674
           1       0.75      0.61      0.67       326

    accuracy                           0.90      2000
   macro avg       0.84      0.78      0.81      2000
weighted avg       0.90      0.90      0.90      2000


===== Naive Bayes =====
Test Accuracy: 0.9000
Cross-Val Accuracy: 0.8931
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.99      0.94      1674
           1       0.90      0.44      0.59       326

    accuracy                           0.90      2000
   macro avg       0.90      0.71      0.76      2000
weighted avg       0.90      0.90      0.89      2000


===== SVM =====
Test Accuracy: 0.9450
Cross-Val Accuracy: 0.9454
Classification Report:
               precision    recall  f1-score   suppor